# Code M Project

Agenda
1. Project Overview
2. Architecture
3. Class hieracy

# Project Overview

# Architecture

In [1]:
import logging
import sys
import requests
import encodings
import re
import jieba

from datetime import datetime
from enum import Enum
from pymongo import MongoClient, DESCENDING, ASCENDING
from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'jieba'

## 1.Configuration classes
全局配置类
    * configuration.py
    * globalvars.py
    * globalutilities.py

### configuration.py
#### Database configurations
```
DB_URI = 'mongodb://localhost' 目标数据库URI
DB_NAME = 'CodeManekineko' 目标数据库名称
```
#### Period execution time
```
PERIOD_MIN = 5 执行周期间隔（分钟）
```

In [4]:
""" To store the global configuration variables """

# Database name
DB_NAME = 'CodeManekineko'
DB_URI='mongodb://localhost'

# Period execution sleep time
PERIOD_MIN = 5


### utilDB
与数据库相关的操作

In [13]:
class EnumDbType(Enum):
    MongoDB = 1,
    DocumentDB = 2

class UtilDb:
    """
    Utility class for MongoDB operation
    Sample of usage:
    util_db = UtilDB('database_name')
    print(util_db.is_exist('articleList',{'title':'sample title'}))
    """
    database_type = None
    client = None
    db = None

    def __init__(self, db_name, db_type=EnumDbType.MongoDB, uri=''):
        """ Class initialization
        keyword arguments:
        db_name: the database name wants to manipulate
        db_type: by default EnumDBType.MongoDB
        uri: the uri for db connection, leave blank for localhost access
        """

        self.database_type = db_type
        self.__db_client_initialization(self.database_type, uri)
        self.db = self._get_db(db_name)

    def __del__(self):
        return

    def __db_client_initialization(self, db_type=EnumDbType.MongoDB, uri=''):
        """ Initialize the databse connection
        keyword arguments:
        db_type: by default is EnumDBType.MongoDB
        uri: by default connect to localhost
        """
        target_uri = uri
        if target_uri == '':
            raise Exception('Configuration item DB_URI is missing in configuration.py.')
            target_uri = 'mongodb://localhost'

        try:
            logging.debug('Request DB connection from %s.' % target_uri)
            self.client = MongoClient(target_uri)

            # Mongo DB on Azure URI demo
            # uri = "mongodb://code-manekineko:woL44kcE6OD3yUbrFuZ7Rv06wsDGRpblJYU1yWek8GDogbhoubho5mhihCgBDDnKAhluofqykKecFN1hAwUQXg==@code-manekineko.documents.azure.com:10250/?ssl=true&ssl_cert_reqs=CERT_NONE"
        except Exception as e:
            # Print error message and halt application(No db connection and nothing can do)
            print('###### App halt during initialize the database connection: %s ######' % e)
            raise

    # Private method
    # To specify the operation dbl
    def _get_db(self, db_name):
        try:
            db = self.client[db_name]
            return db
        except Exception as e:
            print('###### App halt as error occurred: %s ######' % e)
            raise

    # Utility methods
    # ===================

    # Check if record exist in specific collection.
    def is_exist(self, target_collection, filters):
        if self.db[target_collection].find(filters).count() == 0:
            return False
        else:
            return True

    # insert document to specific collection
    def insert(self, target_collection, document):
        self.db[target_collection].insert_one(document)

    def find(self,
             target_collection,
             return_the_first_found=False,
             query_filter=None,
             sort='',
             sort_descending=False,
             page_size=0,
             page_index=0,
             no_cursor_timeout=False):
        """
        Refactored Find operation for MongoDB to replace the find_one, find_all
        :param target_collection: the target collection to manipulate.
        :param return_the_first_found: set to True to just return the first element found, same as find_one operation.
        :param query_filter: dict object, the same usage as MongoDB find query. leave it None to return all.
        :param sort: the column needs to sort, by default is blank, which means no sort needed.
        :param sort_descending: by default is False
        :param page_size: by default no paging, and not applicable when find_all is False
        :param page_index: not applicable when page_size=0
        :param no_cursor_timeout: Set to True to ignore cursor timeout which usually use during long time data processing.
        :return: find_all is True returns Dictionary object, False returns Array object.
        """

        c_target = self.db[target_collection]
        # result = None

        if page_size == 0:
            # No paging, return simple find operation
            if query_filter is None:
                result = c_target.find(no_cursor_timeout=no_cursor_timeout)
            else:
                result = c_target.find(query_filter, no_cursor_timeout=no_cursor_timeout)
        else:
            # No query, find all
            if query_filter is None:
                # Not sort criteria, by default
                if sort == '':
                    result = c_target.find(no_cursor_timeout=no_cursor_timeout).limit(page_size).skip(
                        page_index * page_size)
                else:
                    # Has sort, check sort direction
                    if sort_descending:
                        result = c_target.find(no_cursor_timeout=no_cursor_timeout).sort(sort, DESCENDING).limit(
                            page_size).skip(
                            page_index * page_size)
                    else:
                        result = c_target.find(no_cursor_timeout=no_cursor_timeout).sort(sort, ASCENDING).limit(
                            page_size).skip(
                            page_index * page_size)
            else:
                data = c_target.find(query_filter, no_cursor_timeout=no_cursor_timeout)

                if sort == '':
                    result = data.sort(sort).limit(page_size).skip(page_index * page_size)
                else:
                    if sort_descending:
                        result = data.sort(sort, DESCENDING).limit(page_size).skip(page_index * page_size)
                    else:
                        result = data.sort(sort, ASCENDING).limit(page_size).skip(page_index * page_size)

        if return_the_first_found:
            if len(result) > 0:
                return result[0]
            else:
                return None
        else:
            return result

    def collection_count(self, collection):
        return self.db[collection].count()

    def update(self, collection, query, doc, upsert=False):
        """
        Udate the collection document with parameters
        :param collection: Target collection for update
        :param query: query string
        :param doc: doc for update
        :param upsert: Set to True will insert new one if no match found.
        """
        self.db[collection].update(spec=query, document=doc, upsert=upsert)

    # TODO: Unfinished method
    def delete(self, collection, query):
        self.db[collection].delete_one(query)
        return



### utilLog
与日志相关的操作

In [31]:
# Utility class for logging operation
class UtilLog:
    # Local variables
    _db = None
    _logger = None
    
    def __init__(self, disable_logging_display=True):
        """
        日志工具类，负责所有日志的相关操作
        初始化参数：
        disable_logging_display：是否禁用输出显示而直接保存到日志库，缺省为禁用。
        """
        self._db = UtilDb(db_name=DB_NAME,uri=DB_URI)
        logging.basicConfig(level=logging.DEBUG, format=' %(asctime)s - %(levelname)s - %(message)s')
        logger_name = 'requests'
        if logger_name not in logging.Logger.manager.loggerDict:
            _logger = logging.getLogger(name=logger_name)
            _logger.setLevel(logging.critical)
        
        _logger=logging.getLogger(logger_name)

        if disable_logging_display:
            _logger.disable(logging.debug)
            #logging.disable(logging.DEBUG)

    def __del__(self):
        return

    # Operation log
    def operation_log(self, log, print_to_console=False, debug_only=False):
        """
        记录操作日志
        print_to_console：记录日志时同步打印到输出窗体，缺省为否
        debug_only：仅作调试显示而无须保存到日志数据库，缺省为否。
        """
        if not debug_only:
            log_content = {'timestamp': datetime.now(),
                           'log': log,
                           'log_type': 'Operation'}
            self._db.insert(COL_LOGS, log_content)
            _logger.debug('%s - %s' % (datetime.now(), log))

            if print_to_console:
                print('%s - %s' % (datetime.now(), log.encode('UTF-8').decode('UTF-8', 'ignore')))
        else:
            print('%s - %s' % (datetime.now(), log.encode('UTF-8').decode('UTF-8', 'ignore')))

    # Error log
    def error_log(self, log: '', e: Exception, print_to_console=False, debug_only=False):
        """
        记录错误日志
        print_to_console：记录日志时同步打印到输出窗体，缺省为否
        debug_only：仅作调试显示而无须保存到日志数据库，缺省为否。
        """

        if not debug_only:
            log_content = {'timestamp': datetime.now(),
                           'log': log + ' ' + str(e),
                           'log_type': 'Error'}
            self._db.insert(COL_LOGS, log_content)
            _logger.error("Error occurred: %s" % log + ' ' + str(e))
            if print_to_console:
                # print("%s - Error occurred: %s" %(datetime.now(), log.encode('UTF-8').decode('UTF-8', 'ignore')))
                print("%s - Error occurred: %s" % (datetime.now(), log + ' ' + str(e)))
        else:
            # print("%s - Error occurred: %s" %(datetime.now(), log.encode('UTF-8').decode('UTF-8', 'ignore')))
            print("%s - Error occurred: %s" % (datetime.now(), log + ' ' + str(e)))


### globalvars.py
存储全局变量

```
用于统计的全局变量
TOTAL_FETCHED = 0  获取的记录条数
TOTAL_SAVED = 0  保存的记录条数
TOTAL_IGNORED = 0 忽略的记录条数
TOTAL_ERROR = 0 错误数量

用于存储数据库中的表名，以更易懂且更松散的方式来进行调用。以后即使修改了数据库表名，仅需要更新这里的相关对应即可。
COL_ACCESS_HISTORY = 'UrlAccessHistory'  访问历史表，用于存储所有爬虫抓取的历史URL
COL_BLACKLIST = 'UrlBlackList' 黑名单，及会被跳过的URL
COL_ROUTINE_URL_LIST = 'RoutineUrlList' 巡回URL抓取列表，每次抓取任务都会从该表中获取并逐一执行抓取任务
COL_ARTICLE = 'ArticleBody' 抓取文章的主体内容会保存在这里
COL_ARTICLE_LIST = 'ArticleList' 文章列表，主要是概要信息
COL_LOGS = 'Logs' 日志
COL_STOCK_LIST = 'StockList' 股票列表
COL_NLP_DICT = 'NLP_Dict' 自然语言字典
```

In [14]:
""" To store the global variables """
TOTAL_FETCHED = 0
TOTAL_SAVED = 0
TOTAL_IGNORED = 0
TOTAL_ERROR = 0

COL_ACCESS_HISTORY = 'UrlAccessHistory'
COL_BLACKLIST = 'UrlBlackList'
COL_ROUTINE_URL_LIST = 'RoutineUrlList'
COL_ARTICLE = 'ArticleBody'
COL_ARTICLE_LIST = 'ArticleList'
COL_LOGS = 'Logs'
COL_STOCK_LIST = 'StockList'
COL_NLP_DICT = 'NLP_Dict'


### globalutilities.py
可被全局调用的工具类

#### UTIL_DB
数据库操作工具类，其中的DB_URI和DB_NAME来自于configuration.py
#### UTIL_LOG
日志记录工具类，缺省disable_logging_display=False，如果希望不在调试输出中进行输出而仅保留数据库记录请设置disable_logging_display=True


In [33]:
UTIL_DB = UtilDb(db_name=DB_NAME,uri=DB_URI)
UTIL_LOG = UtilLog(disable_logging_display=False)

## 2. Utility classes

### utilScraping
与爬虫相关的操作

In [36]:
# Utility class for web scraping operation
class UtilScrapingOperation:
    """ Utility class for scraping operation """
    _scraping_rule = None
    _blacklist_urls = None

    def __init__(self, routine_url=None):
        """
        Scraping utility class to help do web scraping operations.
        :param routine_url: the url in routine url list which will binding to specific scraping rules.
        :return:
        """
        # Load url black list
        self._blacklist_urls = UTIL_DB.find(COL_BLACKLIST, query_filter={'status': 'Active'})

        if routine_url is not None:
            self.set_scraping_rule(routine_url)
        return

    def has_visited(self, url:str):
        """
        Check if the specified URL has been visited before.
        :param url: 
        :return: 
        """
        doc = UTIL_DB.find_one(COL_ACCESS_HISTORY,{'url':url})
        if doc is not None and len(doc) > 0:
            if doc['access_datetime'] != '':
                return True
            else:
                return False
        else:
            return False

    def in_blacklist(self, url: str):
        """
        Check if the specified URL in the blacklist
        :param url: Partially or whole in will all return True.
        :return: 
        """
        for black_url in self._blacklist_urls:
            if re.search(black_url['url'], url) is None:
                return False
            else:
                return True

    def set_scraping_rule(self, routine_url):
        """
        Set and initialize the ScrapingRule instance
        :param routine_url:
        :return:
        """
        self._scraping_rule=ScrapingRule(routine_url)

    def get_scraping_rule(self):
        """
        Get the ScrapingRule instance
        :return:
        """
        return self._scraping_rule

    def fetch_urls_in_page(self, bs_obj: BeautifulSoup):
        """
        获取所传入url地址页面中所有url链接并保存进数据库待抓取地址列表
        :param bs_obj: BeautifulSoup object,页面内容
        :return:
        """
        lst_urls = bs_obj.find_all('a', {'target': '_blank', 'href': re.compile('shtml')})
        for url_tag in lst_urls:
            if not self.in_blacklist(url_tag['href']):
                history = UrlAccessHistory(url_tag['href'])
                if not history.has_visited():
                    history.save()
                else:
                    # Remove from the target url list if has been visited.
                    lst_urls.remove(url_tag)
            else:
                # Remove from the target url list if found in blacklist
                UTIL_LOG.operation_log('%s in blacklist, no save.' % url_tag['href'], debug_only=True)
                lst_urls.remove(url_tag)

        return lst_urls

    def unvisited_urls(self):
        """
        从UrlAccessHistory表中返回所有还未访问过的url地址
        :return: list对象
        """
        return UTIL_DB.find(target_collection=COL_ACCESS_HISTORY, query_filter={'access_datetime': ''})

    def parse_and_save_article_content(self, bs_obj: BeautifulSoup):
        """
        根据抓取规则抓取并保存文章内容
        :param bs_obj: 待抓取的BeautifulSoup对象
        :return:
        """
        # TODO: Need to study how to generalize and make the reusable scraping rule model
        scraping_rule = self.get_scraping_rule()

        return

    def get_routine_urls(self):
        """
        Fetch the target url list from database
        """

        # The data structure of collection RoutineUrlList as below
        #    {
        #        "_id" : ObjectId("58ca0635f5e5454a930515a9"),
        #        "url" : "http://finance.sina.com.cn/stock/",
        #        "access_history" : {},
        #        "scraping_rule" : [ 
        #                                    {
        #                                        "field_name" : "title",
        #                                        "tag" : "h1",
        #                                        "property" : "id",
        #                                        "filter" : "artibodyTitle"
        #                                    },
        #                                    {
        #                                        "field_name" : "article_body",
        #                                        "tag" : "div",
        #                                        "property" : "id",
        #                                        "filter" : "articleContent"
        #                                    },
        #                                    {
        #                                        "field_name" : "article_datetime",
        #                                        "tag" : "span",
        #                                        "property" : "class",
        #                                        "filter" : "time-source"
        #                                    },
        #                                    {
        #                                        "field_name" : "article_editor",
        #                                        "tag" : "p",
        #                                        "property" : "class",
        #                                        "filter" : "article-editor"
        #                                    },
        #                                    {
        #                                        "field_name" : "article_summary",
        #                                        "tag" : "",
        #                                        "property" : "",
        #                                        "filter" : ""
        #                                    }
        #                                ],
        #        "active" : true
        #    }

        _start = datetime.now()
        lst_target = UTIL_DB.find(COL_ROUTINE_URL_LIST, {'active': True})
        _duration = datetime.now() - _start
        UTIL_LOG.operation_log('%s - 获得%s待扫描目标地址，本次操作耗时%s。' % (datetime.now(),
                                                               lst_target.count(),
                                                               _duration), True)

        return lst_target

    def fetch_url(self, url: str, force_visit=False, ignore_error=True, check_blacklist=True):
        """ 
        Fetch web content from given url.
        :param url: string, the target url to fetch.
        :param force_visit: force visit even has visited before
        :param ignore_error: Ignore raised error
        :param check_blacklist: Check blacklist before access, bypass if exists in the blacklist.
        :return BeautifulSoup object
        """

        if check_blacklist:
            if self.in_blacklist(url):
                UTIL_LOG.operation_log('%s targeted by blacklist checking, access bypassed.' % url, debug_only=True)
                return

        if not force_visit and self.has_visited(url):
            # Url in the blacklist or has been visited before
            if not force_visit:
                UTIL_LOG.operation_log('%s targeted by visited checking, access bypassed.' % url, debug_only=True)
                return

        # 清洗url格式,用\s匹配任意的空白符，包括空格，制表符(Tab)，换行符，中文全角空格等后替换去除
        url = re.compile(r'\s').sub('', url)

        res = requests.get(url)
        res.encoding = res.apparent_encoding
        UTIL_LOG.operation_log(log='%s, status code:%s' % (url, res.status_code), debug_only=True)

        try:
            res.raise_for_status()
        except Exception as e:
            # 发生错误,返回None
            UTIL_LOG.error_log(e, debug_only=True)
            if not ignore_error:
                raise e
            return None

        bs_obj = BeautifulSoup(res.text, 'lxml')
        return bs_obj

    def fetch_article_list(self, url):
        """ 
        Fetch article list from given url.

        keyword arguments:
        url: string, the url to fetch the article list
        """

        _starttime = datetime.now()
        bs_obj = self.fetch_url(url)

        lst_articles = bs_obj.find_all('a', {'target': '_blank', 'href': re.compile('shtml')})
        for i in lst_articles:
            gap = i['href'].rfind('.shtml') - i['href'].rfind('/') - 1
            if gap < 12 or i.string is None:
                lst_articles.remove(i)

        _executeduration = datetime.now() - _starttime

        UTIL_LOG.operation_log(
            "%s - 从%s获得%s待处理文章项目，本次操作耗时%s" % (datetime.now(), url, len(lst_articles), _executeduration))

        return lst_articles

    def parse_all_url_from_page(self, url):
        """
        从所给的页面中抓取所有超链接
        缺省规则：target=blank，以shtml结尾
        返回：获取到的url列表
        """
        _start = datetime.now()
        bs_obj = self.fetch_url(url)
        lst_urls = bs_obj.find_all('a', {'target': '_blank', 'href': re.compile('shtml')})
        UTIL_LOG.operation_log('在%s找到%s个超链接。本次操作时间%s' % (url,
                                                         lst_urls.count(),
                                                         datetime.now() - _start))
        return lst_urls

    def fetch_next_page_url(self, url):
        """ 
        Find the next page tag and return the url

        keyword arguments:
        url: the url page to find the tag with 下一页 displayed.
        
        return: string, the retrieved url or None if not found.
        """

        _starttime = datetime.now()
        bs_obj = self.fetch_url(url)
        next_url_tag = bs_obj.find('a', title='下一页')
        if next_url_tag is not None:
            next_url = next_url_tag['href']
            if next_url.find('./') == 0:
                next_url1 = url.replace(url[url.rfind('/') + 1:len(url)], next_url[2: len(next_url)])
            _executeduration = datetime.now() - _starttime
            UTIL_LOG.operation_log("%s - 从%s获取下一页跳转链接成功，本次操作耗时%s" % (datetime.now(), url, _executeduration))
            return next_url1

        else:
            _executeduration = datetime.now() - _starttime
            UTIL_LOG.operation_log("%s - 从%s获取下一页跳转链接，没有找到匹配链接。本次操作耗时%s" % (datetime.now(), url, _executeduration))
            return None

    def parse_content_from_bsobj(self, bs_obj, target, parse_filter):
        """ Parse content from inputted BeautifulSoup object

        keyword arguments:
        bs_obj: the BeautifulSoup object to manipulate
        target: the html tag which are looking for
        parse_filter: the rules to match the specific tag
        """
        result = bs_obj.find(target, parse_filter)
        if result is not None:
            return result.get_text()
        else:
            return ''

    def update_routine_history(self, url, doc):
        """ Update the routine history.
        
        keyword arguments:
        url: the url which scraped, used to find the match from target url list.
        doc: the document of target list which added the right history list item.
        """
        UTIL_DB.update(COL_ROUTINE_URL_LIST, {'url': url}, doc)



### utilNLP
与自然语言相关的操作

In [ ]:
class UtilNLP:
    """ Utility class for Nature Language Process """

    def __init__(self):
        # load user dict
        lst_dict = UTIL_DB.find(COL_NLP_DICT, '')
        for i in lst_dict:
            jieba.add_word(i['word'], i['freq'], i['tag'])
        return

    def __delete__(self, instance):
        return

    def process(self, content):
        return jieba.cut(content)


### utilTransformation
与数据转换相关的操作

In [34]:
class UtilTransformation:
    def __init__(self):
        pass

    def sort_document_array(self, array=None, array_to_sort='', ascend=True):
        """
        Sort the input array by dict 
        :param array: the list to manipulate.
        :param array_to_sort: the item name to sort, which must be array type.
        :param ascend: default to True, set False to sort descend.
        :return: the sorted array
        """
        if array is None:
            return None
        else:
            result = []
            l = 0
            for i in array:
                l = len(i[array_to_sort])
                offset = 0
                flag_quit = False

                if len(result) == 0:
                    # Empty list, just append
                    result.append(i)
                    # print_array_order(result,array_to_sort,'append at empty')
                else:

                    for ii in result:
                        len_of_result = len(ii[array_to_sort])
                        # print('comparing array item(%s) > result array(%s):%s' %(l,len_of_result,l>len_of_result))
                        if ascend:
                            if len_of_result > l and not quit:
                                result.insert(offset, i)
                                flag_quit = True
                                # print_array_order(result,array_to_sort,'insert')
                            offset += 1

                    if not flag_quit:
                        # Still no quit means no small array found, just append
                        result.append(i)
                        # print_array_order(result,array_to_sort,'append at last')

            return result

    def print_array_order(self, array=None, array_to_sort='', action=''):
        print('============= %s' % action)
        for i in array:
            print(len(i[array_to_sort]))


## 3. Business logic classes

### 2.1 utilStock
与股票相关的业务操作

In [2]:
import sys
?sys.setrecursionlimit()